# Lesson

## Import

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Code

In [ ]:
def compute_model(y_name, X_name, data):
    y = data.loc[:, y_name]
    X = sm.add_constant(data.loc[:, X_name].values)
    model = sm.OLS(y, X).fit()
    return show_table(model, X_name)

In [ ]:
def show_table(model, X_name):
    index_name = ['Intercept']

    if isinstance(X_name, str):
        index_name.append(X_name)
    elif isinstance(X_name, list):
        index_name = index_name + X_name

    df = pd.read_html(model.summary2().as_html())[1]
    colname = df.iloc[0]
    df = df.rename(columns=df.iloc[0]).drop(0).set_index(np.nan)
    df.index.name = None
    df.index = index_name

    return df

In [ ]:
def compute_VIF(columns, data):
    X = data.loc[:, columns]
    X.loc[:, 'Intercept'] = 1
      
    vif = pd.DataFrame()
    vif.loc[:, 'variables'] = X.columns
    vif.loc[:, 'VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
      
    return vif.drop(vif.tail(1).index)

In [ ]:
advertising = pd.read_csv('data/Advertising.csv')
credit = pd.read_csv('data/Credit.csv')

In [ ]:
# table 3.2
y = advertising['sales']
X = advertising[['TV']]
X = sm.add_constant(X)
slr = sm.OLS(y, X).fit()
rss = slr.resid.std(ddof=X.shape[1])
print(f'RSS: {rss}')
slr.summary().tables[0]

In [ ]:
# table 3.1
compute_model('sales', 'TV', advertising)

In [ ]:
# table 3.3
compute_model('sales', 'radio', advertising)

In [ ]:
# table 3.3
compute_model('sales', 'newspaper', advertising)

In [ ]:
# table 3.4
y = advertising['sales']
X = advertising[['TV', 'radio', 'newspaper']]
X = sm.add_constant(X)
mlr = sm.OLS(y, X).fit()
mlr.summary().tables[1]

In [ ]:
# table 3.6
y = advertising['sales']
X = advertising[['TV', 'radio', 'newspaper']]
X = sm.add_constant(X)
mlr = sm.OLS(y, X).fit()
rss = mlr.resid.std(ddof=X.shape[1])
print(f'RSS: {rss}')
mlr.summary().tables[0]

In [ ]:
# table 3.7
credit_owner_dummy = pd.get_dummies(credit, columns=['Own'])
compute_model('Balance', 'Own_Yes', credit_owner_dummy)

In [ ]:
# table 3.8
credit_region_dummy = pd.get_dummies(credit, columns=['Region'])
compute_model('Balance', ['Region_West', 'Region_South'], credit_region_dummy)

In [ ]:
# table 3.9
advertising['TVxradio'] = advertising['TV'] * advertising['radio']
compute_model('sales', ['TV', 'radio', 'TVxradio'], advertising)

In [ ]:
# table 3.11
groupby_cols = ['model', 'column']

model_1_df = compute_model('Balance', ['Age', 'Limit'], credit)
model_1_df['model'] = 'Model 1'
model_2_df = compute_model('Balance', ['Rating', 'Limit'], credit)
model_2_df['model'] = 'Model 2'

model_df = pd.concat([model_1_df, model_2_df]).reset_index()
model_df.rename(columns = {'index': 'column'}, inplace=True)
model_df = model_df.groupby(groupby_cols, group_keys=True).apply(lambda a: a[:])
model_df[model_df.columns.drop(groupby_cols)]

In [ ]:
compute_VIF(['TV', 'radio', 'newspaper'], advertising)

# Lab

## Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

from ISLP import load_data
from ISLP.models import ModelSpec as MS, summarize, poly

In [ ]:
plt.rcParams['font.family'] = 'Avenir'
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.size'] = 12
plt.rcParams['figure.dpi'] = 300

In [ ]:
IMG_EXPORT_CONFIG = {
    'dpi': 500,
    'bbox_inches': 'tight',
    'pad_inches': 0.15,
}

In [ ]:
sns.set_theme(style='whitegrid')
sns.set_palette('hls', 8)

## Code

The `boston` dataset records `medv` (median house value) for 506 neighborhoods around Boston. 

We will build a regression model to predict `medv` using **13** predictors such as:
- `rmvar` (average number of rooms per house),
- `age` (proportion of owner-occupied units built prior to 1940), and
- `lstat` (percent of households with low socioeconomic status).

In [ ]:
boston_df = load_data('Boston')
boston_df.head()

In [ ]:
boston_df?

### Simple Linear Regression

Our response will be `medv` and `lstat` will be the single predictor.

In [ ]:
X = pd.DataFrame({
    'intercept': np.ones(boston_df.shape[0]),
    'lstat': boston_df['lstat'],
})
X.head()

`sm.OLS()` does not fit the model, rather it specifies the model, and then `model.fit()` does the actual fitting.

In [ ]:
y = boston_df['medv']
slr = sm.OLS(y, X)
slr_result = slr.fit()

In [ ]:
summarize(slr_result)

In [ ]:
slr_result.summary()

In [ ]:
slr_result.params

#### Using Transformations: Fit and Transform

`ModelSpec()` (renamed `MS()` in the preamble) creates a transform object, and then a pair of methods `transform()` and `fit()` are used to construct a corresponding model matrix.

In this simple case, the `fit()` method does very little; it simply checks that the variable `lstat` specified in design exists in `boston`. Then `transform()` constructs the model matrix with two columns: an intercept and the variable `lstat`.

These two operations can be combined with the `fit_transform() `method.

In [ ]:
design = MS(['lstat'])
# option 1
design = design.fit(boston_df)
X = design.transform(boston_df)
# option 2
X = design.fit_transform(boston_df)
X.head()

The `get_prediction()` method can be used to obtain predictions, and produce confidence intervals and prediction intervals for the prediction of `medv` for given values of `lstat`.

In [ ]:
new_df = pd.DataFrame({'lstat': [5, 10, 15]})
new_X = design.transform(new_df)
new_X

In [ ]:
new_pred = slr_result.get_prediction(new_X)
new_pred.predicted_mean

In [ ]:
# confidence interval
new_pred.conf_int(alpha=0.05)

In [ ]:
# prediction internal
new_pred.conf_int(obs=True, alpha=0.05)

The 95% confidence interval associated with an `lstat` value of 10 is (24.47, 25.63), and the 95% prediction interval is (12.82, 37.28).

#### Defining Functions

In [ ]:
def add_linear_line(ax, m, b, *args, **kwargs):
    """ Add a line with slope m and intercept b to ax """
    xlim = ax.get_xlim()
    ylim = [m * xlim[0] + b, m * xlim[1] + b]
    ax.plot(xlim, ylim, *args, **kwargs)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(x='lstat', y='medv', data=boston_df, ax=ax)
add_linear_line(ax, slr_result.params[0], slr_result.params[1], 'r--')

We plot the fitted values (`.fittedvalues`) against theirs residuals (`.resid`). We add a horizontal line at `0` for reference using the `ax.axhline()` method, indicating it should be black (`c='k'`) and have a dashed linestyle (`ls='--'`).

From the plot, there is some evidence of non-linearity.

In [ ]:
ax = plt.subplots(figsize=(8, 8))[1]
ax.scatter(slr_result.fittedvalues, slr_result.resid)
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--')

Various influence measures describing the regression model are computed with the `get_influence()` method.

Leverage statistics can be computed for any number of predictors using the `hat_matrix_diag` attribute of the value returned by the `get_influence()` method.

The `np.argmax()` function identifies the index of the largest element of an array, optionally computed over an axis of the array.

In [ ]:
influence = slr_result.get_influence()
influence.summary_frame()

In [ ]:
ax = plt.subplots(figsize=(8, 8))[1]
ax.scatter(np.arange(X.shape[0]), influence.hat_matrix_diag)
ax.set_xlabel('Index')
ax.set_ylabel('Leverage')
np.argmax(influence.hat_matrix_diag)

### Multiple Linear Regression

In order to fit a multiple linear regression model using least squares, we again use the `ModelSpec()` transform to construct the required model matrix and response.

In [ ]:
X = MS(['lstat', 'age']).fit_transform(boston_df)
mlr = sm.OLS(y, X)
mlr_result = mlr.fit()
summarize(mlr_result)

Now, we will use all columns to predict `medv`.

In [ ]:
terms = boston_df.columns.drop('medv')
terms

In [ ]:
X = MS(terms).fit_transform(boston_df)
mlr = sm.OLS(y, X)
mlr_result = mlr.fit()
summarize(mlr_result)

From the summarize, we see that `indus` and `age` has high $p$-value. So we'd remove these 2 columns from the prediction.

In [ ]:
new_terms = boston_df.columns.drop(['medv', 'indus', 'age'])
new_X = MS(new_terms).fit_transform(boston_df)
new_mlr = sm.OLS(y, new_X)
new_mlr_result = new_mlr.fit()
summarize(new_mlr_result)

### Multivariate Goodness of Fit

#### List Comprehension

**Variance Inflation Factor (VIF):** is the ratio of the variance of $\hat\beta_j$ when fitting the full model divided by the variance of $\hat\beta_j$ if fit on its own.

Function `VIF()` takes 2 arguments: a dataframe or array, and a variable column index.

In [ ]:
vals = [VIF(X, i) for i in range(1, X.shape[1])]
vif = pd.DataFrame({'vif': vals}, index=X.columns[1:])
vif

### Interaction Terms

**Interaction Term:** constructed by computing the product of $X_1$ and $X_2$.

⟹ Extended model: $$Y = \beta_0 + \beta_1X1 + \beta_2X_2 + \beta_3X_1X_2 + \epsilon$$

Including a tuple `('lstat', 'age')` tells the model matrix builder to include an interaction term between `lstat` and `age`.

In [ ]:
X_2 = MS(['lstat', 'age', ('lstat', 'age')]).fit_transform(boston_df)
mlr_2 = sm.OLS(y, X_2)
mlr_result_2 = mlr_2.fit()
summarize(mlr_result_2)